# Graph Neural Network Classification on the PROTEINS Dataset

For the first approach, I'm going to use [Spektral](https://graphneural.network/getting-started/) for Python to build my GCN layer and then perform our classification. 

Spektral is a library for Python for Graph Neural Networks, built on Tensorflow and Keras. 

Our second experiment will be built with PyTorch Geometric.

In [ ]:
# !pip install spektral

In [ ]:
# Reading in the PROTEINS dataset
from spektral.datasets import TUDataset

data = TUDataset('PROTEINS')
data

100%|█████████████████████████████████████████| 447k/447k [00:00<00:00, 854kB/s]


Successfully loaded PROTEINS.


TUDataset(n_graphs=1113)

In [ ]:
# Since we want to utilize the Spektral GCN layer, we want to follow the original paper for this method and perform some preprocessing:
from spektral.transforms import GCNFilter

data.apply(GCNFilter())

In [ ]:
# Split our train and test data. This just splits based on the first 80%/second 20% which isn't entirely ideal, so we'll shuffle the data first.
import numpy as np

np.random.shuffle(data)
split = int(0.8 * len(data))
data_train, data_test = data[:split], data[split:]

In [ ]:
# Spektral is built on top of Keras, so we can use the Keras functional API to build a model that first embeds,
# then sums the nodes together (global pooling), then classifies the result with a dense softmax layer

# First, let's import the necessary layers:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

In [ ]:
# Now, we can use model subclassing to define our model:

class ProteinsGNN(Model):
  
  def __init__(self, n_hidden, n_labels):
    super().__init__()
    # Define our GCN layer with our n_hidden layers
    self.graph_conv = GCNConv(n_hidden)
    # Define our global pooling layer
    self.pool = GlobalSumPool()
    # Define our dropout layer, initialize dropout freq. to .5 (50%)
    self.dropout = Dropout(0.5)
    # Define our Dense layer, with softmax activation function
    self.dense = Dense(n_labels, 'softmax')

  # Define class method to call model on input
  def call(self, inputs):
    out = self.graph_conv(inputs)
    out = self.dropout(out)
    out = self.pool(out)
    out = self.dense(out)

    return out

In [ ]:
# Instantiate our model for training
model = ProteinsGNN(32, data.n_labels)

In [ ]:
# Compile model with our optimizer (adam) and loss function
model.compile('adam', 'categorical_crossentropy')

In [ ]:
# Here's the trick - we can't just call Keras' fit() method on this model.
# Instead, we have to use Loaders, which Spektral walks us through. Loaders create mini-batches by iterating over the graph
# Since we're using Spektral for an experiment, for our first trial we'll use the recommended loader in the getting started tutorial

# TODO: read up on modes and try other loaders later
from spektral.data import BatchLoader

loader = BatchLoader(data_train, batch_size=32)

In [ ]:
# Now we can train! We don't need to specify a batch size, since our loader is basically a generator
# But we do need to specify the steps_per_epoch parameter

model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)

Epoch 1/10
28/28 [==============================] - 2s 29ms/step - loss: 15.5761
Epoch 2/10
28/28 [==============================] - 1s 24ms/step - loss: 5.7475
Epoch 3/10
28/28 [==============================] - 1s 25ms/step - loss: 5.5035
Epoch 4/10
28/28 [==============================] - 1s 22ms/step - loss: 4.6222
Epoch 5/10
28/28 [==============================] - 1s 22ms/step - loss: 4.6596
Epoch 6/10
28/28 [==============================] - 1s 22ms/step - loss: 5.2084
Epoch 7/10
28/28 [==============================] - 1s 22ms/step - loss: 4.8452
Epoch 8/10
28/28 [==============================] - 1s 21ms/step - loss: 5.2131
Epoch 9/10
28/28 [==============================] - 1s 22ms/step - loss: 5.6248
Epoch 10/10
28/28 [==============================] - 1s 21ms/step - loss: 4.6807


In [ ]:
# To evaluate, let's instantiate another loader to test

test_loader = BatchLoader(data_test, batch_size=32)

In [ ]:
# And feed it to our model by calling .load()

loss = model.evaluate(loader.load(), steps=loader.steps_per_epoch)

print('Test loss: {}'.format(loss))

28/28 [==============================] - 1s 15ms/step - loss: 3.5004
Test loss: 3.500354528427124


## PyTorch Geometric GCN

Pytorch Geometric provides [GCN layers](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html) based on Kipf & Welling's original paper: ["Semi-Supervised Classification with Graph Convolutional Networks"](https://arxiv.org/abs/1609.02907) on which I've based the bulk of my research and write-ups.

While my original goal was to use my [original experiment](https://colab.research.google.com/drive/1NUQgUdrdvIddewdQyGEpas_mPaFzC8-e?usp=sharing) (based off of [this](https://towardsdatascience.com/understanding-graph-convolutional-networks-for-node-classification-a2bfdb7aba7b) resource) to build this from scratch, I ran into difficulties trying to embed and classify such a large dataset, specifically with Colab RAM allowances.

For this reason, I sought out different methods and found that this problem had already been solved, and for purposes of time and demonstration chose to delve into Pytorch Geometric rather than invent the wheel. 

In order to successfully learn to implement this approach with this library, I relied on the Pytorch Geometric [documentation](https://pytorch-geometric.readthedocs.io/en/latest/index.html) as well as [this notebook](https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing) written by matthias.fey@tu-dortmund.de. 

I would like to extend thanks and all due credit to these authors, as this work and research would not be possible without them. Further credits and citations can be found in the [README](https://github.com/sidneyarcidiacono/graph-convolutional-networks) of this repository.

In [61]:
# Install required packages.
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

In [63]:
import torch
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='data/TUDataset', name='PROTEINS')

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: PROTEINS(1113):
Number of graphs: 1113
Number of features: 3
Number of classes: 2

Dataset: PROTEINS(1113):
Number of graphs: 1113
Number of features: 3
Number of edges: 162
Average node degree: 3.86
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


In [64]:
torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:150]
test_dataset = dataset[150:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 150
Number of test graphs: 963


In [65]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
Batch(batch=[2534], edge_index=[2, 9562], ptr=[65], x=[2534, 3], y=[64])

Step 2:
Number of graphs in the current batch: 64
Batch(batch=[2599], edge_index=[2, 9512], ptr=[65], x=[2599, 3], y=[64])

Step 3:
Number of graphs in the current batch: 22
Batch(batch=[604], edge_index=[2, 2288], ptr=[23], x=[604, 3], y=[22])



In [66]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(3, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [67]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
      out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
      loss = criterion(out, data.y)  # Compute the loss.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      optimizer.zero_grad()  # Clear gradients.

def test(loader):
  model.eval()

  correct = 0
  for data in loader:  # Iterate in batches over the training/test dataset.
      out = model(data.x, data.edge_index, data.batch)  
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      correct += int((pred == data.y).sum())  # Check against ground-truth labels.
  return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 201):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.6000, Test Acc: 0.5950
Epoch: 002, Train Acc: 0.6000, Test Acc: 0.5950
Epoch: 003, Train Acc: 0.6667, Test Acc: 0.6303
Epoch: 004, Train Acc: 0.6000, Test Acc: 0.5950
Epoch: 005, Train Acc: 0.6000, Test Acc: 0.5950
Epoch: 006, Train Acc: 0.6267, Test Acc: 0.6085
Epoch: 007, Train Acc: 0.7000, Test Acc: 0.6677
Epoch: 008, Train Acc: 0.7000, Test Acc: 0.6739
Epoch: 009, Train Acc: 0.7067, Test Acc: 0.6771
Epoch: 010, Train Acc: 0.7267, Test Acc: 0.6771
Epoch: 011, Train Acc: 0.7067, Test Acc: 0.6739
Epoch: 012, Train Acc: 0.7067, Test Acc: 0.6771
Epoch: 013, Train Acc: 0.7133, Test Acc: 0.6791
Epoch: 014, Train Acc: 0.7200, Test Acc: 0.6833
Epoch: 015, Train Acc: 0.7333, Test Acc: 0.6781
Epoch: 016, Train Acc: 0.6800, Test Acc: 0.6355
Epoch: 017, Train Acc: 0.7533, Test Acc: 0.6854
Epoch: 018, Train Acc: 0.7000, Test Acc: 0.6521
Epoch: 019, Train Acc: 0.6867, Test Acc: 0.6646
Epoch: 020, Train Acc: 0.6733, Test Acc: 0.6490
Epoch: 021, Train Acc: 0.7200, Test Acc: